# Weather Dataset

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
from windrose import WindroseAxes
import matplotlib.cm as cm
import numpy as np

sns.set(rc={'figure.figsize':(13,13)})

## 1. Data Import - Weather Dataset<a id='1'></a>

In [5]:
path = '../data/weather/'
file_name = 'weather.csv'
full_path = path + file_name

weather_df = pd.read_csv(full_path)
weather_df.shape

(331166, 10)

In [6]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331166 entries, 0 to 331165
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   timestamp       331166 non-null  object 
 1   site_id         331166 non-null  object 
 2   airTemperature  331038 non-null  float64
 3   cloudCoverage   160179 non-null  float64
 4   dewTemperature  330838 non-null  float64
 5   precipDepth1HR  197980 non-null  float64
 6   precipDepth6HR  18162 non-null   float64
 7   seaLvlPressure  309542 non-null  float64
 8   windDirection   318161 non-null  float64
 9   windSpeed       330592 non-null  float64
dtypes: float64(8), object(2)
memory usage: 25.3+ MB


Code below to reduce df memory size

In [401]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [402]:
weather_df = reduce_mem_usage(weather_df)

Mem. usage decreased to 10.11 Mb (60.0% reduction)


In [403]:
weather_df.head()

,timestamp,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
0,2016-01-01 00:00:00,Panther,19.40625,NaN,19.40625,0.0,NaN,NaN,0.0,0.0
1,2016-01-01 01:00:00,Panther,21.09375,6.0,21.09375,-1.0,NaN,1019.5,0.0,0.0
2,2016-01-01 02:00:00,Panther,21.09375,NaN,21.09375,0.0,NaN,1019.0,210.0,1.5
3,2016-01-01 03:00:00,Panther,20.59375,NaN,20.00000,0.0,NaN,1018.0,0.0,0.0
4,2016-01-01 04:00:00,Panther,21.09375,NaN,20.59375,0.0,NaN,1019.0,290.0,1.5


## 2. Target Location Selection<a id='2'></a>

In [404]:
weather_df.site_id.value_counts()

Panther     17544
Gator       17544
Fox         17543
Bear        17542
Hog         17542
Rat         17539
Peacock     17539
Eagle       17536
Swan        17535
Bull        17529
Bobcat      17525
Shrew       17516
Robin       17516
Mouse       17516
Wolf        17505
Lamb        17500
Cockatoo    16975
Crow        16860
Moose       16860
Name: site_id, dtype: int64

Similar to the data wrangling process we did for buildings, we want to only pick the weather information we care about: "Moose (Ottawa, Ontario)", "Crow (Ottawa, Ontario)", "Hog (Minneapolis, Minnesota)", "Cockatoo (Cornell, New York)"

In [405]:
target_sites = ['Moose', 'Crow', 'Hog', 'Cockatoo']
weather_df = weather_df[weather_df.site_id.isin(target_sites)]

#replace site_id with real location's name
weather_df = weather_df.replace(to_replace='Moose', value='Ottawa_1')
weather_df = weather_df.replace(to_replace='Crow', value='Ottawa_2')
weather_df = weather_df.replace(to_replace='Hog', value='Minneapolis')
weather_df = weather_df.replace(to_replace='Cockatoo', value='Cornell')

weather_df.shape

(68237, 10)

In [406]:
weather_df.site_id.value_counts()

Minneapolis    17542
Cornell        16975
Ottawa_1       16860
Ottawa_2       16860
Name: site_id, dtype: int64

### 2.1 Location: Ottawa Data Investigation<a id='2.1'></a>

We notice that we have 2 locations as Ottawa. In theory, the weather information under site_ids "Ottawa_1" and 'Ottawa_2' should be the same. The fact that they have same amount of data (16860) is a hint. We will now confirm this. If we have a number of 16860 of duplicate items, then we can confirm that the weather information under both site_ids are the same.

In [407]:
ottawa_df = weather_df.loc[weather_df.site_id.isin(['Ottawa_1', 'Ottawa_2']), :]
ottawa_df

,timestamp,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
122723,2016-01-01 00:00:00,Ottawa_1,-2.199219,NaN,-3.599609,NaN,NaN,1014.5,270.0,2.099609
122724,2016-01-01 01:00:00,Ottawa_1,-2.300781,NaN,-3.800781,NaN,0.0,1014.5,260.0,2.099609
122725,2016-01-01 02:00:00,Ottawa_1,-2.800781,NaN,-4.000000,NaN,NaN,1014.0,260.0,1.500000
122726,2016-01-01 03:00:00,Ottawa_1,-3.000000,NaN,-3.800781,5.0,NaN,1014.0,260.0,1.500000
122727,2016-01-01 04:00:00,Ottawa_1,-3.300781,NaN,-3.800781,NaN,NaN,1013.5,230.0,1.000000
...,...,...,...,...,...,...,...,...,...,...
209036,2017-12-31 19:00:00,Ottawa_2,-21.500000,NaN,-26.906250,NaN,0.0,1030.0,280.0,3.599609
209037,2017-12-31 20:00:00,Ottawa_2,-21.203125,NaN,-26.406250,NaN,NaN,1030.0,290.0,5.101562
209038,2017-12-31 21:00:00,Ottawa_2,-22.296875,NaN,-26.906250,NaN,NaN,1030.0,280.0,3.099609
209039,2017-12-31 22:00:00,Ottawa_2,-22.906250,NaN,-26.796875,NaN,NaN,1030.0,260.0,2.099609


In [408]:
ottawa_df = ottawa_df.drop(columns='site_id')
ottawa_df.duplicated().sum()

16860

This confirms the duplications. We are going to removal all data that associated with site_id "Ottawa_2" and change "Ottawa_1" to "Ottawa"

In [409]:
weather_df.drop(weather_df[weather_df['site_id'] == 'Ottawa_2'].index, inplace=True)
weather_df = weather_df.replace(to_replace='Ottawa_1', value='Ottawa')
weather_df.groupby('site_id')['airTemperature'].count()

site_id
Cornell        16973
Minneapolis    17540
Ottawa         16860
Name: airTemperature, dtype: int64

In [410]:
weather_df.head()

,timestamp,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
122723,2016-01-01 00:00:00,Ottawa,-2.199219,NaN,-3.599609,NaN,NaN,1014.5,270.0,2.099609
122724,2016-01-01 01:00:00,Ottawa,-2.300781,NaN,-3.800781,NaN,0.0,1014.5,260.0,2.099609
122725,2016-01-01 02:00:00,Ottawa,-2.800781,NaN,-4.000000,NaN,NaN,1014.0,260.0,1.500000
122726,2016-01-01 03:00:00,Ottawa,-3.000000,NaN,-3.800781,5.0,NaN,1014.0,260.0,1.500000
122727,2016-01-01 04:00:00,Ottawa,-3.300781,NaN,-3.800781,NaN,NaN,1013.5,230.0,1.000000


In summary, we have 3 locations' weather information: Cornell, Minneapolis, and Ottawa. The fact that they have different counts is telling us we have some missing timeline. We will investigate later.

## 3. Data Understanding <a id='3'></a>

Before we jump into summarizing missing data. We use the information below to develop a understanding of our data. The information is from the data publisher.

- timestamp: date and time in the format YYYY-MM-DD hh:mm:ss. Local timezone.
- site_id: animal-code-name for the site.
- airTemperature: The temperature of the air in degrees Celsius (ºC).
- cloudCoverage: Portion of the sky covered in clouds, in oktas.
- dewTemperature: The dew point (the temperature to which a given parcel of air must be cooled at constant pressure and water 
- vapor content in order for saturation to occur) in degrees Celsius (ºC).
- precipDepth1HR: The depth of liquid precipitation that is measured over a one hour accumulation period (mm).
- precipDepth6HR: The depth of liquid precipitation that is measured over a six hour accumulation period (mm).
- seaLvlPressure: The air pressure relative to Mean Sea Level (MSL) (mbar or hPa).
- windDirection: The angle, measured in a clockwise direction, between true north and the direction from which the wind is blowing (degrees).
- windSpeed: The rate of horizontal travel of air past a fixed point (m/s).

### 3.1 Missing Values by Column<a id='3.1'></a>

In [411]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51377 entries, 122723 to 313649
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   timestamp       51377 non-null  object 
 1   site_id         51377 non-null  object 
 2   airTemperature  51373 non-null  float16
 3   cloudCoverage   16695 non-null  float16
 4   dewTemperature  51324 non-null  float16
 5   precipDepth1HR  22784 non-null  float16
 6   precipDepth6HR  4548 non-null   float16
 7   seaLvlPressure  49964 non-null  float16
 8   windDirection   50487 non-null  float16
 9   windSpeed       51339 non-null  float16
dtypes: float16(8), object(2)
memory usage: 2.0+ MB


In [412]:
def missing_summary(df):
    missing_sum = df.isnull().sum()
    missing_mean = df.isnull().mean() * 100
    
    missing_df = pd.concat([missing_sum,missing_mean], axis=1)
    missing_df.columns =['missing_count', '%']
    missing_df = missing_df.sort_values('missing_count', ascending=False)
    return missing_df

missing_summary(weather_df)

,missing_count,%
precipDepth6HR,46829,91.147790
cloudCoverage,34682,67.504915
precipDepth1HR,28593,55.653308
seaLvlPressure,1413,2.750258
windDirection,890,1.732293
dewTemperature,53,0.103159
windSpeed,38,0.073963
airTemperature,4,0.007786
timestamp,0,0.000000
site_id,0,0.000000
